In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import datetime

import talib as ta

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
#from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
DfMain = pd.read_csv("C:/Users/marli/Downloads/FinalStocksDFHighResolutionMin.csv")
DfMain

,Symbol,Name,Market Cap,Country,IPO Year,Sector,Industry,Unnamed: 0,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,A,Agilent Technologies Inc. Common Stock,4.973733e+10,United States,1999.0,Capital Goods,Electrical Products,2021-05-24 09:30:00-04:00,133.509995,134.119995,132.529999,134.000000,180601.0,0.0,0.0
1,A,Agilent Technologies Inc. Common Stock,4.973733e+10,United States,1999.0,Capital Goods,Electrical Products,2021-05-24 10:30:00-04:00,134.035004,134.410004,133.979996,134.369995,128425.0,0.0,0.0
2,A,Agilent Technologies Inc. Common Stock,4.973733e+10,United States,1999.0,Capital Goods,Electrical Products,2021-05-24 11:30:00-04:00,134.380005,134.410004,133.710007,133.740005,98121.0,0.0,0.0
3,A,Agilent Technologies Inc. Common Stock,4.973733e+10,United States,1999.0,Capital Goods,Electrical Products,2021-05-24 12:30:00-04:00,133.740005,134.000000,133.740005,133.940002,112184.0,0.0,0.0
4,A,Agilent Technologies Inc. Common Stock,4.973733e+10,United States,1999.0,Capital Goods,Electrical Products,2021-05-24 13:30:00-04:00,133.970001,134.050003,133.895004,133.990005,83879.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5488391,ZYXI,Zynex Inc. Common Stock,5.070879e+08,United States,NaN,Health Care,Biotechnology: Electromedical & Electrotherape...,2021-11-22 11:30:00-05:00,14.420000,14.440000,14.180000,14.250000,17490.0,0.0,0.0
5488392,ZYXI,Zynex Inc. Common Stock,5.070879e+08,United States,NaN,Health Care,Biotechnology: Electromedical & Electrotherape...,2021-11-22 12:30:00-05:00,14.270000,14.510000,14.270000,14.430000,21602.0,0.0,0.0
5488393,ZYXI,Zynex Inc. Common Stock,5.070879e+08,United States,NaN,Health Care,Biotechnology: Electromedical & Electrotherape...,2021-11-22 13:30:00-05:00,14.420000,14.520000,14.400000,14.490000,12756.0,0.0,0.0
5488394,ZYXI,Zynex Inc. Common Stock,5.070879e+08,United States,NaN,Health Care,Biotechnology: Electromedical & Electrotherape...,2021-11-22 14:30:00-05:00,14.445000,14.630000,14.440000,14.540000,14734.0,0.0,0.0


In [3]:
# Importing libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from  sklearn.ensemble import GradientBoostingClassifier
import multiprocessing
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score,precision_score,recall_score

In [12]:
def DataPrep(DF,Stock):#Filters to desired rows and add needed stock info
    Df = DF[DF["Symbol"]==Stock]#Filter to only selected stock
    Df = Df.drop(Df[Df['Volume'] == 0].index)#Droping mins where no stocks were exchanged
    #Df = Df.rename(columns={'Unnamed: 0': 'Datetime'}) 
    Df['Datetime'] = pd.to_datetime(Df['Datetime'])#making date time info the right data type
    Df = Df.drop(["IPO Year"], axis=1)#NA for most stocks
    Df.set_index('Datetime', inplace=True)
    n = 10#Number of days smoothed
    Df['RSI'] = ta.RSI(np.array(Df['Close'].shift(1)), timeperiod=n)#Generate stock info. read paper for details
    Df['SMA'] = Df['Close'].shift(1).rolling(window=n).mean()
    Df['Corr'] = Df['Close'].shift(1).rolling(window=n).corr(Df['SMA'].shift(1))
    Df['SAR'] = ta.SAR(np.array(Df['High'].shift(1)),
                   np.array(Df['Low'].shift(1)), 0.2, 0.2)
    Df['ADX'] = ta.ADX(np.array(Df['High'].shift(1)), np.array(
    Df['Low'].shift(1)), np.array(Df['Open']), timeperiod=n)
    Df['OpenShifted'] = Df['Open'].shift(1)
    Df['CloseShifted'] = Df['Close'].shift(1)
    Df['HighShifted'] = Df['High'].shift(1)
    Df['LowShifted'] = Df['Low'].shift(1)
    Df['OO'] = Df['Open']-Df['OpenShifted']
    Df['OC'] = Df['Open']-Df['CloseShifted']
    Df['Ret'] = (Df['Open'].shift(-1)-Df['Open'])/Df['Open']
    for i in range(1, n):
        Df['return%i' % i] = Df['Ret'].shift(i)
    #Trimming the dataDF
    Df.loc[Df['Corr'] < -1, 'Corr'] = -1#limiting correlation to reasnable values
    Df.loc[Df['Corr'] > 1, 'Corr'] = 1
    Df = Df.dropna()
    return(Df)

In [5]:
def DataSplit(DF,t=.8):#Split the data into x and y
    split = int(t*len(DF))
    Df = DF
    #Creating output signal
    Df['Signal'] = 0#
    Df.loc[Df['Ret'] > Df['Ret'][:split].quantile(q=0.66), 'Signal'] = -1 #Converting problem to
    Df.loc[Df['Ret'] < Df['Ret'][:split].quantile(q=0.33), 'Signal'] = 1 #classification problem
    #Df.loc[Df['Ret'] > Df['Ret'][:split].quantile(q=0.8), 'Signal'] = 2
    #Df.loc[Df['Ret'] < Df['Ret'][:split].quantile(q=0.2), 'Signal'] = -2#Experemented with 5 catagorys
    
    # Assign a value of 0 to 'Signal' column at 1529 time
    Df.loc[(Df.index.hour == 15) & (Df.index.minute == 29), 'Signal'] = 0
    # Assign a value of 0 to 'Fut_Ret' column at 1529 time
    #Df.loc[(Df.index.hour == 15) & (Df.index.minute == 29), 'Fut_Ret'] = 0
    split = int(t*len(Df))
    TestDf = Df.iloc[split:]#Test/Training data needs to be split in time.
    TrainDf = Df.iloc[:split]#Cant have autocorrelation from future datapoints taint model
    return(TrainDf,TestDf)

def XYPrep(DF):
    #Selecting needed values
    X = DF.drop(['Close', 'Signal', 'High','Open',
            'Low', 'Ret',"Name","Dividends", "Country",'Market Cap','Volume', "Symbol",
             "Sector","Industry","Stock Splits"], axis=1)
    Y = DF['Signal']
    Ret = DF['Ret']
    return(X,Y,Ret)

In [6]:
def TrainFit(X_Train,Y_Train,Model,parameters):
    #training model with given parts
    steps = [('scaler', StandardScaler()), ('Model', Model)]
    pipeline = Pipeline(steps)

    clf = GridSearchCV(pipeline,parameters, cv=5 , verbose=0,n_jobs=-1,scoring = "f1_macro")#Not sure if this is best metric
    clf.fit(X_Train, Y_Train)
    #print(clf.best_score_)
    #print(clf.best_params_)
    pipeline.set_params(**clf.best_params_)
    pipeline.fit(X_Train, Y_Train)
    return(pipeline)

#Testing model with key relationships
def TestModel(Mod,X_Test,Y_Test,Ret_Test,Name = "Model"):
    predictions = Mod.predict(X_Test)
    #cm = confusion_matrix(Y_Test, predictions, labels=Mod.classes_)

    #disp = ConfusionMatrixDisplay(confusion_matrix=cm,
    #                           display_labels=Mod.classes_)
    #disp.plot()
    StragyReturn = Ret_Test*predictions
    accuracy = accuracy_score(Y_Test, predictions)          
    recall = recall_score(Y_Test,predictions,average=None)
    precision = precision_score(Y_Test,predictions,average=None)
    DF = pd.DataFrame(np.array([[StragyReturn.mean(), recall[0], recall[1],recall[1]
                                 , precision[0], precision[1],precision[1],accuracy]]),
                   columns=[f"{Name} StragyReturn", f"{Name} recall -1", f"{Name} recall 0",f"{Name} recall 1",
                            f"{Name} precision -1 ",f"{Name} precision 0",f"{Name} precision 1",f"{Name} accuracy"])
    return(DF)

In [7]:
def Driver(DfMain,StockName):
    Df = DataPrep(DfMain, StockName)
    TrainDf,TestDf = DataSplit(Df)
    X_Test,Y_Test,Ret_Test = XYPrep(TestDf)
    X_Train,Y_Train,Ret_Train = XYPrep(TrainDf)
    
    #Train XGB model
    TrainedModel = TrainFit(X_Train,Y_Train,Model = GradientBoostingClassifier(), parameters = {'Model__max_depth': [1,2,4], 
              'Model__n_estimators': [50, 100],
               'Model__min_weight_fraction_leaf': [.1],
              'Model__learning_rate': [.9]})
    #Test XGB model
    Results1=TestModel(TrainedModel,X_Test,Y_Test,Ret_Test,Name = "XGB")
    
    #Train Random Forest model
    TrainedModel2 = TrainFit(X_Train,Y_Train,Model = RandomForestClassifier(), 
                             parameters = {'Model__n_estimators': [100, 200,400]})
    #Test XGB Random Forest model
    Results2=TestModel(TrainedModel2,X_Test,Y_Test,Ret_Test,Name = "RandomForest")
    
    #Train KNN model
    TrainedModel3 = TrainFit(X_Train,Y_Train,Model = KNeighborsClassifier(), 
                             parameters = {'Model__n_neighbors': [1, 5,20]})
    #Test KNN model
    Results3 = TestModel(TrainedModel3,X_Test,Y_Test,Ret_Test,Name = "KNN")
    
    #Train Stacking model
    TrainedModel4 = TrainFit(X_Train,Y_Train,Model = StackingClassifier(estimators=[('XGB', GradientBoostingClassifier()),
                                                                                    ('KNN', KNeighborsClassifier())],
                                                                        final_estimator=LogisticRegression()), 
                             parameters = {'Model__XGB__max_depth': [1,2,4], 
                            'Model__XGB__n_estimators': [50, 100],
                            'Model__XGB__min_weight_fraction_leaf': [.1],
                            'Model__XGB__learning_rate': [.9],
                            'Model__KNN__n_neighbors': [1, 5,20]})
    
    #Test Stacking model
    Results4 = TestModel(TrainedModel4,X_Test,Y_Test,Ret_Test,Name = "Stacking")
    
    
    #Selecting usfull info on the stock
    DFMainInfo = Df.head(1)[["Symbol","Name","Market Cap","Country","Sector","Industry"]]
    
    #Removing datetime index
    DFMainInfo.reset_index(drop=True, inplace=True)
    #Returning all usfull analytics of the analysis
    ReturnDF = pd.concat([DFMainInfo, Results1, Results2, Results3,Results4], axis=1)
    return(ReturnDF)

In [13]:
Driver(DfMain, "MSFT")
#Df = DataPrep(DfMain, "MSFT")

AttributeError: 'Index' object has no attribute 'hour'

In [9]:
#from sklearn.metrics import SCORERS
#sorted(SCORERS.keys())

In [10]:
#raise Exception
StockNameDF = pd.read_csv("C:\\Users\\marli\\Downloads\\nasdaq_screener_1637446085891.csv")

StockNames = StockNameDF["Symbol"]

#NewDF = pd.read_csv("C:\\Users\\marli\\Downloads\\FinalAnalysisHour1.csv")

#StockNamesAlreadyDone = list(NewDF["Symbol"])

new_strings = []
for string in StockNames:
    new_string = string.replace("/", "")
    new_strings.append(new_string)
StockNames = new_strings
i=0
try:
    for Name in StockNames:
        print("Started working on {}".format(Name))
        #if (Name in StockNamesAlreadyDone):
        #    print("Already Done")
        #    continue
        start = time.time()
        #if(i%5!=2):
        #    continue
        try:
            if(i==0):
                MainDF2 = Driver(DfMain, Name)
                i+=1
                print(i)
                print(MainDF2[["Symbol","Stacking accuracy"]])
                end = time.time()
                print(end - start)
                continue
        except Exception as e:
            print(e)
            continue
        i+=1
        print(i)
        try:
            Biz = Driver(DfMain, Name)
        except Exception as e:
            print(e)
            continue
        print(Biz[["Symbol","Stacking accuracy"]])
        MainDF2 = pd.concat([MainDF2, Biz], axis=0)
        end = time.time()
        print(end - start)
except Exception as e :
    print(e)
    MainDF2.to_csv("C:\\Users\\marli\\Downloads\\FinalAnalysisHour2.csv")
    raise e
MainDF2.to_csv("C:\\Users\\marli\\Downloads\\FinalAnalysisHour2.csv")

Started working on A
'Datetime'
Started working on AA
'Datetime'
Started working on AAC
'Datetime'
Started working on AACG
'Datetime'
Started working on AACI
'Datetime'
Started working on AACIU
'Datetime'
Started working on AACIW
'Datetime'
Started working on AADI
'Datetime'
Started working on AAIC
'Datetime'
Started working on AAIC^B
'Datetime'
Started working on AAIC^C
'Datetime'
Started working on AAIN
'Datetime'
Started working on AAL
'Datetime'
Started working on AAMC
'Datetime'
Started working on AAME
'Datetime'
Started working on AAN
'Datetime'
Started working on AAOI
'Datetime'
Started working on AAON
'Datetime'
Started working on AAP
'Datetime'
Started working on AAPL
'Datetime'
Started working on AAQC
'Datetime'
Started working on AAT
'Datetime'
Started working on AATC
'Datetime'
Started working on AAU
'Datetime'
Started working on AAWW
'Datetime'
Started working on AB
'Datetime'
Started working on ABB
'Datetime'
Started working on ABBV
'Datetime'
Started working on ABC
'Date

KeyboardInterrupt: 

In [ ]:
MainDF2

In [ ]:

#MainDF2 = pd.concat([MainDF2, NewDF], axis=0)

In [ ]:
len(set(MainDF2["Symbol"]))

In [ ]:
#MainDF2.to_csv("C:\\Users\\marli\\Downloads\\FinalAnalysisMin1.csv")

In [ ]:
#split = int(.8*len(Df))
#Df['Ret'].hist(bins=200)
#plt.axvline(x=Df['Ret'][:split].quantile(q=0.66),color='r')
#plt.axvline(x=Df['Ret'][:split].quantile(q=0.33),color='r')
#lt.axvline(x=Df['Ret'][:split].quantile(q=0.8),color='r')
#plt.axvline(x=Df['Ret'][:split].quantile(q=0.2),color='r')
#Y_Test.value_counts()

In [ ]:
#TrainedModel2 = TrainFit(X_Train,Y_Train,Model = SVC(), parameters = {'Model__C': [10, 100, 1000, 10000], 
#                                                'Model__gamma': [1e-2, 1e-1, 1e0], 'Model__kernel': ['rbf']})
#TestModel(TrainedModel2,X_Test,Y_Test,Ret_Test)

In [ ]:
#TrainedModel3 = TrainFit(X_Train,Y_Train,Model = RandomForestClassifier(), parameters = {'Model__n_estimators': [100, 200]})
#TestModel(TrainedModel3,X_Test,Y_Test,Ret_Test)

In [ ]:
#TrainedModel4 = TrainFit(X_Train,Y_Train,Model = XGBClassifier(), parameters = {'Model__random_state': [123], "Model__verbosity":[0], 'Model__use_label_encoder':[False]})
#TestModel(TrainedModel4,X_Test,Y_Test,Ret_Test)

In [ ]:
#TrainedModel5 = TrainFit(X_Train,Y_Train,Model = LogisticRegression(), parameters = {'Model__random_state': [123]})
#TestModel(TrainedModel5,X_Test,Y_Test,Ret_Test)